# AI Parse Document Processor and Chunker for Vector Indexing

This notebook uses Databricks AI parse functions to process PDF files from a Unity Catalog volume, extracts content using AI, chunks the text, and stores the results in a Delta table for vector indexing.

## Features:
- Uses AI parse functions for intelligent document processing
- Processes PDF files from Unity Catalog volume
- Extracts text and structure using AI
- Chunks text for optimal vector indexing
- Stores results in Delta table with metadata
- Handles multiple files in batch processing


## 1. Configuration


In [ ]:
# Configuration
SOURCE_VOLUME_PATH = "/Volumes/vbdemos/dbdemos_autoloader/raw_data/usecase-planning-agent-pdf/"
DESTINATION_TABLE_NAME = "vbdemos.usecase_agent.usecase_planning_agent_pdf_parsed"
CHUNKS_TABLE_NAME = "vbdemos.usecase_agent.pdf_chunks"

# AI Parse parameters
PARSE_EXTENSIONS = ['.pdf', '.jpg', '.jpeg', '.png']
LIMIT = 1000  # Maximum number of files to process
PARTITION_COUNT = 4  # Number of partitions for processing

# Processing options
ENABLE_CHUNKING = False  # Set to False if using models with large context windows (e.g., 272k tokens)
CHUNK_SIZE = 500  # characters per chunk (only used if ENABLE_CHUNKING=True)
CHUNK_OVERLAP = 50  # overlap between chunks (only used if ENABLE_CHUNKING=True)

print(f"Source volume path: {SOURCE_VOLUME_PATH}")
print(f"Destination table: {DESTINATION_TABLE_NAME}")
print(f"Chunks table: {CHUNKS_TABLE_NAME}")
print(f"Chunking enabled: {ENABLE_CHUNKING}")
if ENABLE_CHUNKING:
    print(f"Chunk size: {CHUNK_SIZE} characters")
else:
    print("Using full documents (no chunking) - suitable for large context window models")


## 2. AI Parse Document Processing


In [ ]:

# Create the AI parse query
ai_parse_query = f"""
CREATE OR REPLACE TABLE {DESTINATION_TABLE_NAME} AS (
-- Parse documents with ai_parse
WITH all_files AS (
  SELECT
    path,
    content
  FROM
    READ_FILES('{SOURCE_VOLUME_PATH}', format => 'binaryFile')
  ORDER BY
    path ASC
  LIMIT {LIMIT}
),
repartitioned_files AS (
  SELECT *
  FROM all_files
  -- Force Spark to split into partitions
  DISTRIBUTE BY crc32(path) % {PARTITION_COUNT}
),
-- Parse the files using ai_parse document
parsed_documents AS (
  SELECT
    path,
    ai_parse_document(content) as parsed
  FROM
    repartitioned_files
  WHERE array_contains(array{tuple(PARSE_EXTENSIONS)}, lower(regexp_extract(path, r'(\\.[^.]+)$', 1)))
),
raw_documents AS (
  SELECT
    path,
    null as raw_parsed,
    decode(content, "utf-8") as text
  FROM 
    repartitioned_files
  WHERE NOT array_contains(array{tuple(PARSE_EXTENSIONS)}, lower(regexp_extract(path, r'(\\.[^.]+)$', 1)))
),
-- Extract page markdowns from ai_parse output
sorted_page_contents AS (
  SELECT
    path,
    page:content AS content
  FROM
    (
      SELECT
        path,
        posexplode(try_cast(parsed:document:pages AS ARRAY<VARIANT>)) AS (page_idx, page)
      FROM
        parsed_documents
      WHERE
        parsed:document:pages IS NOT NULL
        AND CAST(parsed:error_status AS STRING) IS NULL
    )
  ORDER BY
    page_idx
),
-- Concatenate so we have 1 row per document
concatenated AS (
    SELECT
        path,
        concat_ws('\\n\\n', collect_list(content)) AS full_content
    FROM
        sorted_page_contents
    GROUP BY
        path
),
-- Bring back the raw parsing since it could be useful for other downstream uses
with_raw AS (
    SELECT
        a.path,
        b.parsed as raw_parsed,
        a.full_content as text
    FROM concatenated a
    JOIN parsed_documents b ON a.path = b.path
)
-- Recombine raw text documents with parsed documents
SELECT *  FROM with_raw
UNION ALL 
SELECT * FROM raw_documents
)
"""

print("AI Parse query created. Executing...")
spark.sql(ai_parse_query)

print(f"✅ AI Parse processing completed!")
print(f"📊 Table created: {DESTINATION_TABLE_NAME}")

# Display a sample from the table
print("\n📝 Sample data from parsed documents:")
spark.sql(f"""
    SELECT
        path,
        LEFT(text, 200) as text_preview,
        LENGTH(text) as text_length
    FROM {DESTINATION_TABLE_NAME}
    LIMIT 5
""").show(truncate=False)
